In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from prophet import Prophet 
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import performance_metrics, cross_validation

In [3]:
df = pd.read_csv('../clean_data/BRISTOR_Zegoland_all_%.csv')

In [9]:
df_train = df[df['Date'] < '2024-01-01']
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 0 to 25
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             26 non-null     object 
 1   Data type           26 non-null     object 
 2   Measure             26 non-null     object 
 3   Product             26 non-null     object 
 4   Date                26 non-null     object 
 5   Value               26 non-null     float64
 6   demandA             26 non-null     float64
 7   demandB             26 non-null     float64
 8   demandCompA         26 non-null     float64
 9   DemandShareA        26 non-null     float64
 10  newPatientShareA    26 non-null     float64
 11  newPatientShareB    26 non-null     float64
 12  totalPatientShareA  26 non-null     float64
 13  totalPatientShareB  26 non-null     float64
 14  voiceShareA         26 non-null     float64
 15  voiceShareB         26 non-null     float64
dtypes: float64(11),

In [12]:
df_test = df[df['Date'] >= '2024-01-01']
print(df_test.info())


<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 26 to 30
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             5 non-null      object 
 1   Data type           5 non-null      object 
 2   Measure             5 non-null      object 
 3   Product             5 non-null      object 
 4   Date                5 non-null      object 
 5   Value               5 non-null      float64
 6   demandA             5 non-null      float64
 7   demandB             5 non-null      float64
 8   demandCompA         5 non-null      float64
 9   DemandShareA        5 non-null      float64
 10  newPatientShareA    5 non-null      float64
 11  newPatientShareB    5 non-null      float64
 12  totalPatientShareA  5 non-null      float64
 13  totalPatientShareB  5 non-null      float64
 14  voiceShareA         5 non-null      float64
 15  voiceShareB         5 non-null      float64
dtypes: float64(11),